<a href="https://colab.research.google.com/github/alibehnaz/CovidModel/blob/main/covid_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This code uses data from early 2020 to August 2021 to model the behaviour of different Covid variants. Using this model, we assess how much different strategies can be effective to control the spread of the virus and ultimtely move towards a Covid-free world**

**Two Models have been implemented in this code:**

*   SIR Model for explaining how an infectious disease disseminates in the community
*   Preditive model to forecast reproduction rate given various scenarios

In [3]:
# import libraries
from scipy.integrate import odeint
import numpy as np
import pandas as pd

In [4]:
#set formatting for numbers in table
pd.set_option('float_format', '{:f}'.format)
pd.options.display.float_format = "{:.1f}".format

# **SIR Model**
The SIR model is one of the compartmental models, and many models are derivatives of this basic form. The model consists of three compartments:

1.  S(t) is used to represent the individuals not yet infected with the disease at time t, or those susceptible to the disease of the population.
2.  I(t) denotes the individuals of the population who have been infected with the disease and are capable of spreading the disease to those in the susceptible category.
3.  R(t) is the compartment used for the individuals of the population who have been infected and then removed from the disease, either due to immunization or due to death. Those in this category are not able to be infected again or to transmit the infection to others.

This model is reasonably predictive for infectious diseases that are transmitted from human to human, and where recovery confers lasting resistance, such as measles, mumps and rubella.

These variables (S, I, and R) represent the number of people in each compartment at a particular time. To represent that the number of susceptible, infectious and removed individuals may vary over time (even if the total population size remains constant), we make the precise numbers a function of t (time): S(t), I(t) and R(t). For a specific disease in a specific population, these functions may be worked out in order to predict possible outbreaks and bring them under control.









In [5]:
# SIR = Susceptible, Infected, Recovered 
def SIR(rhat,I0,N):
    def deriv(y, t, N, beta, gamma):
        S, I, R = y
        dSdt = -beta * S * I / N
        dIdt = beta * S * I / N - gamma * I
        dRdt = gamma * I
        return dSdt, dIdt, dRdt
    
    #N =  population
    beta = rhat # infected person infects 1 other person per day
    D = 5.0 # infections lasts 7 days
    # reproduction number R₀, the total number of people an infected person infects. 
    gamma = 1.0 / D
    S0, I0, R0 = N-1, I0 , 0  # initial conditions: one infected, rest susceptible
    
    t = np.linspace(0, 29, 30) # Grid of time points (in days)
    y0 = S0, I0, R0 # Initial conditions vector

    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(N, beta, gamma))
    S, I, R = ret.T
    return (I)

In [45]:
# function to measure euclidean distance of two list
def distance(x1,x2):
    y = np.subtract(x1, x2)
    return np.sum(y**2)  

# function to calculate rhat (reproduction rate) for each time series
def rhat_calculate(x1):
    rhat0=1
    rhat = 1
    diff=1
    break_var=10
    while break_var > 0:
        x2=SIR(rhat,x1[0],1000000)
        diff = distance(x1,x2)
        rhat0=rhat0/2
        rhat = rhat + np.sign(sum(np.subtract(x1, x2)))*(rhat0)
        x2=SIR(rhat,x1[0],1000000)
        break_var = break_var -1
    return rhat

Input data is collected using the time series data available since the begining of the pandemic. Time-series data are split into 30-day periods. Each 30-day sample capture covid cases in a country/city where a specific strategy has been in place. Using SIR model, reproduction rate for each 30-day sample is calculated. 


Data Link: https://github.com/owid/covid-19-data
Data for Australia are downloaded separately from state data repositories

In [35]:
# load the dataset
url = 'https://raw.githubusercontent.com/alibehnaz/CovidModel/main/CovidTimeSeries.csv'
data = pd.read_csv(url)

In [36]:
data

,Day,cases,Sample,Cooutries/States,Mask Mandate/Hygiene,Variant,Vax rate,Lockdown,Period
0,1,30,1,Sydney,2,3,20%,3,Aug-21
1,2,36,1,Sydney,2,3,20%,3,Aug-21
2,3,22,1,Sydney,2,3,20%,3,Aug-21
3,4,31,1,Sydney,2,3,20%,3,Aug-21
4,5,16,1,Sydney,2,3,20%,3,Aug-21
...,...,...,...,...,...,...,...,...,...
625,26,1074,21,CA,1,3,70%,2,Jul-21
626,27,537,21,CA,1,3,70%,2,Jul-21
627,28,796,21,CA,1,3,70%,2,Jul-21
628,29,912,21,CA,1,3,70%,2,Jul-21


In [37]:
#sample_size = data["Sample"].max()
n= data["Sample"].max() + 1
for i in range(1,n):
    data_temp = data.loc[data["Sample"]==i]
    x1 = data_temp["cases"].tolist()
    print(rhat_calculate(x1))

0.2724609375
0.3037109375
0.2822265625
0.1103515625
0.1064453125
0.0283203125
0.1220703125
0.3017578125
0.2236328125
0.1455078125
0.2724609375
0.2666015625
0.2568359375
0.1494140625
0.2548828125
0.2197265625
0.1982421875
0.3134765625
0.1884765625
0.1669921875
0.2705078125


# **Predictive model to forecast reproduction rate**

The model will predict reproduction rate given different scenarios.
Independant variables are created manually by investigating for each country and period what strategy has been in action at the time. Independent varaiables are: 
* Restrictions (Masks, lockdowns, movement, travel, work)
* Disease variant
* Vax rate

In [38]:
# load the dataset
url = 'https://raw.githubusercontent.com/alibehnaz/CovidModel/main/CovidReproduction.csv'
data_pre  = pd.read_csv(url)

In [39]:
X = data_pre[['Variant','Mask Mandate/Hygiene', 'Vax rate','Lockdown']]
y = data_pre['R']

In [40]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [41]:
print("Intercept: ", regr.intercept_)
print("Coefficients:", regr.coef_)

Intercept:  0.23424250748921563
Coefficients: [ 0.01486879  0.04804634 -0.07932584 -0.05377755]


In [43]:
y_pred_regr= regr.predict(X)

In [44]:
from sklearn import metrics
meanAbErr = metrics.mean_absolute_error(y, y_pred_regr)
meanSqErr = metrics.mean_squared_error(y, y_pred_regr)
rootMeanSqErr = np.sqrt(metrics.mean_squared_error(y, y_pred_regr))
print('R squared: {:.2f}'.format(regr.score(X,y)*100))
print('Mean Absolute Error:', meanAbErr)
print('Mean Square Error:', meanSqErr)
print('Root Mean Square Error:', rootMeanSqErr)

R squared: 14.24
Mean Absolute Error: 0.05902284051319854
Mean Square Error: 0.0050582110284012475
Root Mean Square Error: 0.0711211011472773
